In [1]:
import cProfile
import lysis
import math
import numpy as np

In [2]:
e = lysis.util.Experiment(r'../../data', experiment_code='2022-12-27-1100')

In [3]:
p = {'total_time': 1}
e.initialize_macro_param(p)
# e.read_file()

In [4]:
macro = lysis.MacroscaleRun(e)

In [5]:
macro.run()

100%|█████████████████████████████████████████████████████████████████████████████| 2920/2920 [00:05<00:00, 523.12it/s]

Total binds: 11,029
Timesteps with changes to degrade time: 723
Total moves: 21,196,167
Total macro unbinds: 0
Total micro unbinds: 19


In [7]:
current_time = 2920 * macro.exp.macro_params.time_step
current_time

0.9997907233333332

In [9]:
macro.m_fiber_status = macro.fiber_status[macro.location]
macro.unbind_by_degradation(macro.bound & (macro.m_fiber_status < current_time), current_time)
macro.unbind_by_time(macro.bound & (macro.binding_time < current_time), current_time)

should_bind = (~macro.bound
               & (macro.binding_time < current_time)
               & (macro.m_fiber_status > current_time)
               & (macro.waiting_time < current_time))

move_chance = np.ones(macro.exp.macro_params.total_molecules, dtype=np.double)
move_chance[~macro.bound] = macro.rng.random(np.count_nonzero(~macro.bound), dtype=np.double)
should_move = move_chance < macro.exp.macro_params.moving_probability
conflict = should_bind & should_move
threshold = np.full(macro.exp.macro_params.total_molecules, -1, dtype=np.double)
threshold[conflict] = ((current_time - macro.binding_time[conflict])
                       / macro.exp.macro_params.time_step)
should_bind[conflict] = macro.rng.random(np.count_nonzero(conflict)) > threshold[conflict]
should_move[conflict] = ~should_bind[conflict]

macro.bind(should_bind, current_time)

In [10]:
still_stuck_to_fiber = np.full(macro.exp.macro_params.total_molecules, False, dtype=np.bool_)
still_stuck_to_fiber[should_bind] = macro.waiting_time[should_bind] > current_time
np.logical_and(still_stuck_to_fiber[should_bind], macro.unbound_by_degradation[should_bind], out=still_stuck_to_fiber[should_bind])
np.count_nonzero(still_stuck_to_fiber)

0

In [11]:
for n in np.arange(macro.exp.macro_params.total_molecules)[still_stuck_to_fiber]:
    print(n)

In [15]:
a = 499
b = 500
c = a*e.macro_params.time_step
d = b*e.macro_params.time_step

In [16]:
%timeit a < b

36.1 ns ± 0.269 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [17]:
%timeit c < d

33.1 ns ± 0.401 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [ ]:
lysis.from_fortran_edge_index(51, 4, 5)

In [2]:
rng = np.random.default_rng(seed=489489496)

In [34]:
a = rng.random(60)

In [35]:
b = rng.integers(60, size=(3, 4))
b

array([[55,  4, 45, 32],
       [ 7, 45, 37, 50],
       [23,  5, 33, 13]], dtype=int64)

In [58]:
a[b]

array([[0.17398308, 0.53022389, 0.28751422, 0.55931457],
       [0.95012645, 0.28751422, 0.51085294, 0.34155978],
       [0.08039694, 0.24990649, 0.24432482, 0.63829939]])

In [36]:
a[b] < 0.5

array([[ True, False,  True, False],
       [False,  True, False,  True],
       [ True,  True,  True, False]])

In [49]:
c = np.sort(a[b] < 0.5, axis=1)

In [50]:
c

array([[False, False,  True,  True],
       [False, False,  True,  True],
       [False,  True,  True,  True]])

In [48]:
np.take_along_axis(a[b], c[:,::-1], axis=1)

array([[0.28751422, 0.17398308, 0.55931457, 0.53022389],
       [0.34155978, 0.28751422, 0.51085294, 0.95012645],
       [0.24432482, 0.24990649, 0.08039694, 0.63829939]])

In [73]:
d = np.count_nonzero(c, axis=1)
d

array([2, 2, 3], dtype=int64)

In [71]:
e = d[np.newaxis, :]
np.ix_(,d)

(array([[0]], dtype=int64), array([[2, 2, 3]], dtype=int64))

In [74]:
a[b][np.full(3, True), d]

array([0.28751422, 0.51085294, 0.63829939])

In [81]:
%timeit np.any(a < 0.1)

8.26 µs ± 108 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [83]:
a = np.empty((20, 9), dtype=int)

In [84]:
a[:] = np.arange(9)

In [85]:
a

array([[0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6, 7, 8]])

In [86]:
a[4,6]=100
a

array([[  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5, 100,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3,   4,   5,   6,   7,   8],
       [  0,   1,   2,   3, 